## Object detection.

CV에서 가장 유용한 경우. Labeling Scenes에서부터 다방면에 걸쳐 쓰인다.

recognition은 second level of Object detection. multiple object within img 확인. 

Template Matching: object detection의 기본. 비유하자면 사막에서 바늘 찾기.

### Finding Waldo (Quickly find a specific pattern within image)

input image를 받은 다음 grayscale로 변환. template에 해당하는 작은 이미지도 받은 다음 gray 변환.

matchtemplate 함수를 이용함. 아래 예시의 경우 match method는 cv2.TM_CCOEFF를 사용. correlation coefficient를 말함

* 링크에 올라간 함수들을 보면, 물론 특정 함수가 다른 함수보다 잘 match하는 경우는 있어도 전체적으로는 유사한 결과를 내는 편

return값은 array이고, 이 값의 bounding box를 반환하는 함수가 cv2.minMaxLoc 함수.

bounding box를 찾은 뒤, 원본그림에 box를 실제로 그린 것.

### Notes on Template Matching

There are a variety of methods to perform template matching, but in this case we are using the correlation coefficient which is specified by the flag **cv2.TM_CCOEFF.**

So what exactly is the cv2.matchTemplate function doing?
Essentially, this function takes a “sliding window” of our waldo query image and slides it across our puzzle image from left to right and top to bottom, one pixel at a time. Then, for each of these locations, we compute the correlation coefficient to determine how “good” or “bad” the match is. 

Regions with sufficiently high correlation can be considered “matches” for our waldo template.
From there, all we need is a call to cv2.minMaxLoc on Line 22 to find where our “good” matches are.
That’s really all there is to template matching!

http://docs.opencv.org/2.4/modules/imgproc/doc/object_detection.html      

In [1]:
import cv2
import numpy as np

# Load input image and convert to grayscale
image = cv2.imread('./MasteringComputerVision-V1.03/Master OpenCV/images/WaldoBeach.jpg')
cv2.imshow('Where is Waldo?', image)
cv2.waitKey(0)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Load Template image
template = cv2.imread('./MasteringComputerVision-V1.03/Master OpenCV/images/waldo.jpg',0)

result = cv2.matchTemplate(gray, template, cv2.TM_CCOEFF)
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

#Create Bounding Box
top_left = max_loc
bottom_right = (top_left[0] + 50, top_left[1] + 50)
cv2.rectangle(image, top_left, bottom_right, (0,0,255), 5)

cv2.imshow('Where is Waldo?', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

그런데, 원본 이미지를 잘라낸 특정 이미지를 사용하는 경우 몇 가지 limitation이 있다.

* scale, rotation, another angle(perspective), brightness / contrast / hue 등에 전부 무력함. distortion에도 취약

이런 문제를 해결하려는 노력으로 "image Features"가 있다.

image features = 'summaries of pictures'. 

"interesting areas that are somewhat unique to the specific image." 

보통 key point features / interest points라고도 불린다.

#### 중요한 이유??

analyse, describe, match image에서 상당히 중요한 기능을 하기 때문. 

image alignment (파노라마 switching = 이미지 매칭하기 등) / 3D reconstruction, robot navigation, object recognition, motion tracking 등등 다양한 분야에 응용된다.

**"interesting"의 정의??**

= distinct, unique한 정보를 담고 있는 부분이라고 이해하면 쉽다. High change of intensities, Corners of edges 등등.

주의할 점: 노이즈도 appear 'informative' when it's not. 그래서 cleaning 작업이 필요.


#### interesting features의 특징

* repeatable : 사진 내 여러 곳에서 비슷하게 발견됨
* distinctive : 각각의 feature는 unique, different to other features of same scene
* compactness / efficiency : 해당 이미지의 특징을 잡아낼 수 있는 최소한의 feature를 말함. 
* locality : features occupies a small area of image, and is robust to clutter & occlusion.



1. Corners as Features.

window shift시 intensity 차이가 크다. 
 - edge : change in one direction (상하 / 좌우)
 - corner : change in all direction (상하좌우 전부 다)


### Harris Corner Detection

#### Harris Corner Detection is an algorithm developed in 1998 for corner detection  (http://www.bmva.org/bmvc/1988/avc-88-023.pdf) and works fairly well.

**cv2.cornerHarris**(input image, block size, ksize, k)
- Input image - should be grayscale and float32 type.
- blockSize - the size of neighborhood considered for corner detection
- ksize - aperture parameter of Sobel derivative used.
- k - harris detector free parameter in the equation
- **Output** – array of corner locations (x,y)

In [1]:
import cv2
import numpy as np

# Load image then grayscale
image = cv2.imread('./MasteringComputerVision-V1.03/Master OpenCV/images/chess.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# The cornerHarris function requires the array datatype to be float32
gray = np.float32(gray)

harris_corners = cv2.cornerHarris(gray, 3, 3, 0.05)

#We use dilation of the corner points to enlarge them\
kernel = np.ones((7,7),np.uint8)
harris_corners = cv2.dilate(harris_corners, kernel, iterations = 2)

# Threshold for an optimal value, it may vary depending on the image.
image[harris_corners > 0.025 * harris_corners.max() ] = [255, 127, 127]

cv2.imshow('Harris Corners', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Improved Corner Detection using - Good Features to Track

In [4]:
import cv2
import numpy as np

img = cv2.imread('./MasteringComputerVision-V1.03/Master OpenCV/images/chess.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# We specific the top 50 corners
corners = cv2.goodFeaturesToTrack(gray, 100, 0.01, 15)
# 마지막 값은 설명하기 복잡하지만 'corner 사이의 최소 거리' 같은 개념으로 이해하면 쉽다.
for corner in corners:
    x, y = corner[0]
    x = int(x)
    y = int(y)
    cv2.rectangle(img,(x-10,y-10),(x+10,y+10),(0,255,0), 2)
    
cv2.imshow("Corners Found", img)
cv2.waitKey()
cv2.destroyAllWindows()

**cv2.goodFeaturesToTrack**(input image, maxCorners, qualityLevel, minDistance)

- Input Image - 8-bit or floating-point 32-bit, single-channel image.
- maxCorners – Maximum number of corners to return. If there are more corners than are found, the strongest of them is returned.
- qualityLevel – Parameter characterizing the minimal accepted quality of image corners. The parameter value is multiplied by the best corner quality measure (smallest eigenvalue). The corners with the quality measure less than the product are rejected. For example, if the best corner has the quality measure = 1500, and the  qualityLevel=0.01 , then all the corners with the quality - - measure less than 15 are rejected.
- minDistance – Minimum possible Euclidean distance between the returned corners.


### 여전히 몇 가지 문제점을 안고 있음

장점: tolerent of "rotation, translation (image shift), slight brightness / affine intensity change에 강하다

단점: large change in intensity or photometric change(brightness, contrast, affine intensity), scaling (enlargen, shrinking)에 약함

이미지 크기가 작아지면 corner를 제대로 찾지 못하고, 이미지 크기가 커지면 corner 수가 지나치게 많아진다. (detect edges로 선회하는 문제점)


## 문제점을 해결하는 몇 가지 알고리즘들

**SIFT** (scale invariant feature transform)

scaling tolerent한 알고리즘이며, CV에서 보편적으로 쓰이는 알고리즘이자 scale invariance issue에서 디폴트로 사용되는 알고리즘. 단, no longer freely available with openCV 3.0+. (patent issue) academic / research purpose로 사용될 경우는 무료이지만 openCV 2.143을 써야 한다고 함.


간단설명.

1. interesting key point를 Difference of Gaussian method라는 방법으로 찾아낸다. 이 방법은 variation이 특정 threshold를 초과하는 경우를 탐지하며, edge descriptor보다 성능이 좋다고 함
2. create vector descriptor for these interesting areas.
    - 이 과정에서 several different scale로 scan을 진행한다. 이걸 하는 이유는 specific stability criteria를 찾아내기 위해서고, 이렇게 찾아낸 criteria를 vector descriptor로 인코딩함. 따라서, 크기에 상관없이 가장 stable scale을 사용하기에 scale invariant 특성을 갖게 됨
    
3. rotation invariance는 Orientation Assignment of key point를 찾아서 해결. image gradient magnitudes라는 걸 사용한다고 하며, 2D direction만 알게 되면 normalize direction이 가능하다는 논리라고 하는데 뭔소린지 잘 모르겠음



**SURF**

speed version of SIFT. Difference of Gaussian의 연산량이 많은 편이라 시간이 오래 걸림. 이 알고리즘은 Hessian Matrix를 사용해 시간단축에 성공했다고 함. 또한 sum of Haar wavelet response for orientation assignment.


SIFT, SURF는 patented이기에 돈이 든다. 그래서 사용할 수 있는 대안들이 몇 있음

**Features from Accelerated Segment Test (FAST) **

key point detection only.(No descriptor. descriptor 계산하려면 유료 알고리즘 필요) Real time application에서 사용한다고 함

**Binary Robust Independent Elementary Features (BRIEF)**

Computers descriptors quickly (유료 알고리즘보다 빠름).

**Oriented FAST and Rotated BRIEF(ORB)**

FAST와 BRIEF를 합친 알고리즘이며 openCV에서 무료로 제공한다.


---


프로세스 절차를 요약하자면
1. Create Detector
2. Input image into detector
3. obtain key points
4. Draw key points.

In [5]:
import cv2
import numpy as np

image = cv2.imread('./MasteringComputerVision-V1.03/Master OpenCV/images/input.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#Create SIFT Feature Detector object
sift = cv2.SIFT()

#Detect key points
keypoints = sift.detect(gray, None)
print("Number of keypoints Detected: ", len(keypoints))

# Draw rich key points on input image
image = cv2.drawKeypoints(image, keypoints, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv2.imshow('Feature Method - SIFT', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

AttributeError: module 'cv2' has no attribute 'SIFT'

In [3]:
import cv2
import numpy as np

image = cv2.imread('./MasteringComputerVision-V1.03/Master OpenCV/images/input.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#Create SURF Feature Detector object
surf = cv2.SURF()

# Only features, whose hessian is larger than hessianThreshold are retained by the detector
surf.hessianThreshold = 500
keypoints, descriptors = surf.detectAndCompute(gray, None)
print("Number of keypoints Detected: ", len(keypoints))

# Draw rich key points on input image
image = cv2.drawKeypoints(image, keypoints, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv2.imshow('Feature Method - SURF', image)
cv2.waitKey()
cv2.destroyAllWindows()

AttributeError: module 'cv2' has no attribute 'SURF_create'

In [2]:
# FAST

import cv2
import numpy as np

image = cv2.imread('./MasteringComputerVision-V1.03/Master OpenCV/images/input.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Create FAST Detector object
fast = cv2.FastFeatureDetector_create()

# Obtain Key points, by default non max suppression is On
# to turn off set fast.setBool('nonmaxSuppression', False)
keypoints = fast.detect(gray, None)
print("Number of keypoints Detected: ", len(keypoints))

# Draw rich keypoints on input image
image = cv2.drawKeypoints(image, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv2.imshow('Feature Method - FAST', image)
cv2.waitKey()
cv2.destroyAllWindows()

Number of keypoints Detected:  8960


In [11]:
# BRIEF

import cv2
import numpy as np

image = cv2.imread('./MasteringComputerVision-V1.03/Master OpenCV/images/input.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Create FAST detector object
fast = cv2.FastFeatureDetector_create()

# Create BRIEF extractor object
# brief = cv2.DescriptorExtractor_create("BRIEF")
brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()
# Determine key points
keypoints = fast.detect(gray, None)

# Obtain descriptors and new final keypoints using BRIEF
# 여기서 descriptor를 만들 수 있다. brief.compute 명령어로.
keypoints, descriptors = brief.compute(gray, keypoints)
print("Number of keypoints Detected: ", len(keypoints))

# Draw rich keypoints on input image
image = cv2.drawKeypoints(image, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
                                    
cv2.imshow('Feature Method - BRIEF', image)
cv2.waitKey()
cv2.destroyAllWindows()

Number of keypoints Detected:  8735


In [15]:
# ORB

import cv2
import numpy as np

image = cv2.imread('./MasteringComputerVision-V1.03/Master OpenCV/images/input.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Create ORB object, we can specify the number of key points we desire
# key point의 최대값을 객체 생성 시 부여할 수 있다. 부여하지 않을 경우 디폴트는 500으로 설정. 500개의 best key point를 찾는 것.
orb = cv2.ORB_create(5000)

# Determine key points
keypoints = orb.detect(gray, None)

# Obtain the descriptors
keypoints, descriptors = orb.compute(gray, keypoints)
print("Number of keypoints Detected: ", len(keypoints))

# Draw rich keypoints on input image
image = cv2.drawKeypoints(image, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv2.imshow('Feature Method - ORB', image)
cv2.waitKey()
cv2.destroyAllWindows()

Number of keypoints Detected:  4993
